Some tests

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
from time import time
from copy import deepcopy

import numpy as np
np.random.seed(2016)

In [3]:
import sys
sys.path.append('../common')
from utils import weight3 as weight_fn, weight_by_index
from utils import bag_weight, score
from utils import MAX_WEIGHT, AVAILABLE_GIFTS, GIFT_TYPES, N_TYPES, N_BAGS

In [4]:
fixed_weights = {}
# for i, g in enumerate(GIFT_TYPES):
#     fixed_weights[g] = weight_fn(i, 5000) - 0.15


# for i, t in enumerate(GIFT_TYPES):
#     only_one_type_state = []
#     state = list([0]*9)
#     state[i] = 1
#     for i in range(AVAILABLE_GIFTS[t]):
#         only_one_type_state.append(state)
#     s = score(only_one_type_state)
#     print("Type : ",t, " | Score : ", s, ", Score/nb : ", s / AVAILABLE_GIFTS[t])

fixed_weights['ball'] = 1.99876912083
fixed_weights['bike'] = 20.0021364556
fixed_weights['blocks'] = 11.6630321858
fixed_weights['book'] = 2.00086596571
fixed_weights['coal'] = 23.7866257713
fixed_weights['doll'] = 4.9993625282
fixed_weights['gloves'] = 1.40310067709
fixed_weights['horse'] = 4.99527064522
fixed_weights['train'] = 10.0234458084
fixed_weights

{'ball': 1.99876912083,
 'bike': 20.0021364556,
 'blocks': 11.6630321858,
 'book': 2.00086596571,
 'coal': 23.7866257713,
 'doll': 4.9993625282,
 'gloves': 1.40310067709,
 'horse': 4.99527064522,
 'train': 10.0234458084}

In [5]:
total_weight = 0
for g in GIFT_TYPES:
    total_weight += fixed_weights[g] * AVAILABLE_GIFTS[g]
total_weight

50511.0646006388

In [6]:
def mean_std(state, count=100):
    w = []
    for c in range(count):
        m = 0
        for i, v in enumerate(state):
            if v > 0:
                for j in range(v):
                    m += weight_by_index(i)
        w.append(m)
    return np.mean(w), np.std(w)


def mean_n_sigma(state, n=0.0):
    m, s = mean_std(state)
    return m + n*s


mean_std((0,0,0,0,1,1,0,0,0)), mean_n_sigma((0,0,0,0,1,1,0,0,0))

((28.553722377229636, 17.352693993448447), 29.147358850484014)

In [7]:
for i, g in enumerate(GIFT_TYPES):
    print(i, g, fixed_weights[g])

0 ball 1.99876912083
1 bike 20.0021364556
2 blocks 11.6630321858
3 book 2.00086596571
4 coal 23.7866257713
5 doll 4.9993625282
6 gloves 1.40310067709
7 horse 4.99527064522
8 train 10.0234458084


In [16]:
decreasing_weight_indices = [
    4, 1, 2, 8, 5, 7, 3, 0, 6 
]

empty_state = [0,0,0,0,0,0,0,0,0]
available_gifts = [AVAILABLE_GIFTS[g] for g in GIFT_TYPES]
goal_score = 0.80 * MAX_WEIGHT

Fill bags

In [17]:
AVAILABLE_GIFTS

{'ball': 1100,
 'bike': 500,
 'blocks': 1000,
 'book': 1200,
 'coal': 166,
 'doll': 1000,
 'gloves': 200,
 'horse': 1000,
 'train': 1000}

In [18]:
def is_available(state, available_gifts):
    for i, t in enumerate(state):
        if t > available_gifts[i]:
            return False
    return True


def next_state(state, gift_index, available_gifts):
    prob_state = list(state)
    while gift_index < len(GIFT_TYPES)-1:
        index = decreasing_weight_indices[gift_index]
        prob_state[index] += 1        
        if is_available(prob_state, available_gifts):
            return prob_state, gift_index
        else:
            prob_state = list(state)
            gift_index += 1
            continue
    return None, gift_index

In [19]:
def fill_bag(state, gift_index, available_gifts, verbose=0):

    state_find_counter = 0 
    best_state = None
    best_weight = 0

    while state_find_counter < 500:
        state_find_counter += 1
        weight = score((state,))
        max_weight_limit = mean_n_sigma(state, n=0.75)
        if weight > best_weight:
            best_state = state
            best_weight = weight

        if verbose > 1:
            print(state_find_counter, "weight : ", weight, " | max_weight_limit : ", max_weight_limit, " | state: ", state)
        if weight < goal_score and max_weight_limit < MAX_WEIGHT:
            prev_state = list(state)
            # Get a next state :
            state, gift_index = next_state(prev_state, gift_index, available_gifts)
            if state is None:
                if verbose > 1:
                    print("=> Return best state under goal_score : gift_index=", gift_index)
                    state = best_state
                break

            if verbose > 1:
                print("=> weight < goal_score : ", prev_state, "->", state)

        elif max_weight_limit >= MAX_WEIGHT:
            if verbose > 1:
                print("=> max_weight_limit >= MAX_WEIGHT : ", state, "->", prev_state)        
            state = list(prev_state)

            # Get a next state :
            gift_index += 1
            state, gift_index = next_state(prev_state, gift_index, available_gifts)
            if state is None:
                if verbose > 1:
                    print("=> Return best state under goal_score : gift_index=", gift_index)
                state = best_state
                break

            if verbose > 1:
                print("=> max_weight_limit >= MAX_WEIGHT : ", prev_state, "->", state)

        elif weight >= goal_score and max_weight_limit < MAX_WEIGHT:
            break
        else:
            print(state_find_counter, "weight : ", weight, " | max_weight_limit : ", max_weight_limit, " | state: ", state)
            raise Exception("WTF")

    if verbose > 0:
        print("Found state: ", state, " : ", score((state,), return_rejected=True))
    return state


def update_available_gifts(available_gifts, state):
    for i, t in enumerate(state):
        assert available_gifts[i] - t >= 0, "Found state is not available : {}, {}".format(state, available_gifts)
        available_gifts[i] = available_gifts[i] - t

        
state = list(empty_state)
gift_index = 0
fill_bag(state, gift_index, available_gifts, 2)        

1 weight :  0.0  | max_weight_limit :  0.0  | state:  [0, 0, 0, 0, 0, 0, 0, 0, 0]
=> weight < goal_score :  [0, 0, 0, 0, 0, 0, 0, 0, 0] -> [0, 0, 0, 0, 1, 0, 0, 0, 0]
2 weight :  25.624763635  | max_weight_limit :  36.558197073  | state:  [0, 0, 0, 0, 1, 0, 0, 0, 0]
=> weight < goal_score :  [0, 0, 0, 0, 1, 0, 0, 0, 0] -> [0, 0, 0, 0, 2, 0, 0, 0, 0]
3 weight :  12.7633948608  | max_weight_limit :  61.0551581247  | state:  [0, 0, 0, 0, 2, 0, 0, 0, 0]
=> max_weight_limit >= MAX_WEIGHT :  [0, 0, 0, 0, 2, 0, 0, 0, 0] -> [0, 0, 0, 0, 1, 0, 0, 0, 0]
=> max_weight_limit >= MAX_WEIGHT :  [0, 0, 0, 0, 1, 0, 0, 0, 0] -> [0, 1, 0, 0, 1, 0, 0, 0, 0]
4 weight :  16.308263334  | max_weight_limit :  55.8633405293  | state:  [0, 1, 0, 0, 1, 0, 0, 0, 0]
=> max_weight_limit >= MAX_WEIGHT :  [0, 1, 0, 0, 1, 0, 0, 0, 0] -> [0, 0, 0, 0, 1, 0, 0, 0, 0]
=> max_weight_limit >= MAX_WEIGHT :  [0, 0, 0, 0, 1, 0, 0, 0, 0] -> [0, 0, 1, 0, 1, 0, 0, 0, 0]
5 weight :  18.3511805951  | max_weight_limit :  46.981338375

[0, 0, 0, 0, 1, 0, 0, 0, 0]

In [ ]:
available_gifts = [AVAILABLE_GIFTS[g] for g in GIFT_TYPES]
total_state = []

empty_state = [0,0,0,0,0,0,0,0,0]

goal_score = 0.80 * MAX_WEIGHT
print("Goal score : ", goal_score)

In [ ]:
count = 1000
while len(total_state) < N_BAGS and count > 0:
    
    # Generate a state
    state = list(empty_state)
    gift_index = 0
    result = fill_bag(state, gift_index, available_gifts, 1)            
    total_state.append(result)
    update_available_gifts(available_gifts, result)    
    
    count -= 1    

In [ ]:
available_gifts, len(total_state)

In [ ]:
score(total_state, return_rejected=True), score(total_state) / len(total_state)

### Try random state generation

In [21]:
def gen_random_state(n=1, state=None):
    state = list([0] * len(GIFT_TYPES)) if state is None else state
    indices = np.random.randint(len(GIFT_TYPES), size=n)
    for index in indices:
        state[index] += 1 if np.random.rand() > 0.5 else -1
        state[index] = max(0, state[index])
    return state

state = None

In [22]:
state = gen_random_state(state=state)
state, score((state,))

([0, 0, 1, 0, 0, 0, 0, 0, 0], 11.595618123166147)

In [26]:
def fill_bag(state, available_gifts, verbose=0):

    state_find_counter = 0 
    best_state = None
    best_weight = 0

    while state_find_counter < 500:
        state_find_counter += 1
        weight = score((state,))
        if verbose > 1:
            print(state_find_counter, "weight : ", weight, " | state: ", state)

        if weight > best_weight:
            best_state = state
            best_weight = weight

        state = gen_random_state(state=state)

        if weight >= goal_score:
            best_state = state
            best_weight = weight
            break

    if verbose > 0:
        print("Found state: ", best_state, " : ", score((best_state,), return_rejected=True))
    return best_state

state = list(empty_state)
fill_bag(state, available_gifts, 2)        

1 weight :  0.0  | state:  [0, 0, 0, 0, 0, 0, 0, 0, 0]
2 weight :  0.0  | state:  [0, 0, 0, 0, 0, 0, 0, 0, 0]
3 weight :  0.0  | state:  [0, 0, 0, 0, 0, 0, 0, 0, 0]
4 weight :  0.0  | state:  [0, 0, 0, 0, 0, 0, 0, 0, 0]
5 weight :  0.0  | state:  [0, 0, 0, 0, 0, 0, 0, 0, 0]
6 weight :  24.7040819319  | state:  [0, 0, 0, 0, 1, 0, 0, 0, 0]
- best_state <- [0, 0, 0, 0, 1, 0, 0, 0, 0]
7 weight :  21.1754977593  | state:  [0, 0, 0, 0, 1, 0, 1, 0, 0]
8 weight :  24.489207862  | state:  [0, 0, 0, 0, 1, 0, 1, 0, 0]
9 weight :  22.5101136655  | state:  [0, 0, 0, 0, 1, 0, 0, 0, 0]
10 weight :  24.3176898165  | state:  [0, 0, 0, 0, 1, 0, 0, 0, 0]
11 weight :  19.7914976461  | state:  [0, 0, 0, 0, 2, 0, 0, 0, 0]
12 weight :  11.4653775027  | state:  [0, 0, 0, 0, 2, 1, 0, 0, 0]
13 weight :  6.6744011254  | state:  [0, 1, 0, 0, 2, 1, 0, 0, 0]
14 weight :  10.0513681992  | state:  [0, 1, 0, 0, 2, 1, 0, 0, 0]
15 weight :  5.074278713  | state:  [0, 1, 0, 0, 2, 1, 0, 0, 0]
16 weight :  9.52622978785  |

KeyboardInterrupt: 